# Exploratory data analysis of the Lego datasets

## Foreword

This notebook will be showing some exploratory data analysis for the `LEGO` dataset located [here](https://www.kaggle.com/rtatman/lego-database). This document will provide you with an idea of what a typical project report should look like for this course. Here I am analyzing the `LEGO` dataset. However, for the project, you will be analyzing the `Disney` dataset. More information about that is available on the course `canvas` page.

# Introduction

## Question(s) of interests
In this analysis, I will be investigating a question associated with the collection of lego datasets.
I am interested in finding out which theme has the most sets associated with it. This is interesting because, as you may know, the lego sets are based on various themes. It would be interesting to see if the themes with the most sets would match popular culture. I would expect the **Star Wars** theme to have the most sets.

## Dataset description 

The below descripitions were taken directly from the [website](https://www.kaggle.com/rtatman/lego-database) where the datasets were obtained.

"LEGO is a popular brand of toy building bricks. They are often sold in sets with in order to build a specific object. Each set contains a number of parts in different shapes, sizes and colors. This database contains information on which parts are included in different LEGO sets. It was originally compiled to help people who owned some LEGO sets already figure out what other sets they could build with the pieces they had."

The lego dataset is composed of $8$ tables, `colors.csv`, `inventories.csv`, `inventory_parts.csv`, `inventory_sets.csv`, `part_categories.csv`, `parts.csv`,`sets.csv` and `themes.csv` . Each table is stored in a `.csv` file and contains different information about lego pieces including shapes, sizes, sets, colors, and themes. I will be using the `sets` and `themes` tables formally described below:

* **sets.csv**
    * This file contains information on LEGO sets, including a unique ID number, the name of the set, the year it was released, its theme and how many parts it includes.
* **themes.csv**
    * This file includes information on lego themes. Each theme is given a unique ID number, a name, and (if it's part of a bigger theme) which theme it's part of.

# Methods and Results

Since I am only interested in computing the themes with the most sets, I will need to use tables that contain information on themes and sets. This implies that I will need to use the **sets** and the **themes** tables.

However, before moving further, let us import the tables and do some basic visualizations.

In [3]:
# Lets import all the required libraries needed for this analysis
import altair as alt
import pandas as pd

# import all the required files
sets = pd.read_csv("data/sets.csv")
themes = pd.read_csv("data/themes.csv")

Lets see what the tables look like.

In [4]:
sets.head()

,set_num,name,year,theme_id,num_parts
0,00-1,Weetabix Castle,1970,414,471
1,0011-2,Town Mini-Figures,1978,84,12
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,2
3,0012-1,Space Mini-Figures,1979,143,12
4,0013-1,Space Mini-Figures,1979,143,12


In [5]:
themes.head()

,id,name,parent_id
0,1,Technic,NaN
1,2,Arctic Technic,1.0
2,3,Competition,1.0
3,4,Expert Builder,1.0
4,5,Model,1.0


Lets get some other information about the **sets** table.

In [6]:
sets.info()
print(sets['set_num'].dtype)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11673 entries, 0 to 11672
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   set_num    11673 non-null  object
 1   name       11673 non-null  object
 2   year       11673 non-null  int64 
 3   theme_id   11673 non-null  int64 
 4   num_parts  11673 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 456.1+ KB
object


The sets table has $11673$ rows and $5$ columns. Every **theme_id** has a **set number**, a **name**, the **year** it was released in and the number of **parts** it has. 

Lets get some other information about the **theme** table.

In [7]:
themes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         614 non-null    int64  
 1   name       614 non-null    object 
 2   parent_id  503 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 14.5+ KB


The **themes** table has $614$ rows with $3$ columns. Every theme has an **id**, a **name**, and a **parent_id**. The **parent_id** is provided only for themes that are a part of a bigger theme. However, for this analysis, only themes that are self contained are considered. 

As a first visualization, lets look at the average number of parts released in each year. To do this, I will use the **sets** table. I will group by year and then compute the average number of parts for each year.

In [8]:
# group by year and compute the average number of parts.
sets_year_group = pd.DataFrame(sets.groupby('year')['num_parts'].mean().sort_values(ascending=False))

# Reset the index so we can plot using altair
sets_year_group = sets_year_group.reset_index()
sets_year_group

,year,num_parts
0,2017,260.820946
1,2016,253.077181
2,2006,246.833922
3,2008,231.598854
4,2007,227.595016
...,...,...
61,1956,18.500000
62,1953,16.500000
63,1959,16.250000
64,1954,12.357143


Now that we have it in the proper format, we can generate a bar plot to visualize it.

In [9]:
# Use altair to generate a bar plot
num_parts_plot = (
    alt.Chart(sets_year_group, width=500, height=300)
    .mark_bar()
    .encode(
        x=alt.X("year:O", title="Release Year"),
        y=alt.Y("num_parts:Q", title="Average number of parts"),
    )
    .properties(title="Average number of LEGO parts released by year")
)
num_parts_plot

alt.Chart(...)

From the above plot, there appears to be an increasing trend in the number of parts over the years. There are, however, some years that had an abnormally high number of parts. For example, `1960` had a much higher average number of parts than its neighboring years. Another interesting phenomenon happens between `1996` and `2004` and between `2010` and `2014`. The average number of parts had a drastic decrease in the average number of parts. 

As a second visualization, lets take a look at the number of themes that are a part of a bigger theme. To do this, only themes with out a missing **(NAN)** value in the **parent_id** column will be considered.

In [10]:
# extract rows with out NAN values in the 'parent_id' column using the drop.na() function
# themes_with_parents = themes[themes['parent_id'].notnull()]
themes_with_parents = themes.dropna(subset=["parent_id"])
themes_with_parents

,id,name,parent_id
1,2,Arctic Technic,1.0
2,3,Competition,1.0
3,4,Expert Builder,1.0
4,5,Model,1.0
5,6,Airport,5.0
...,...,...,...
608,609,The LEGO Batman Movie,535.0
610,611,Series 17 Minifigures,535.0
611,612,Star Wars Episode 8,158.0
612,613,Freemakers,158.0


Now lets group by name and count the frequency of the 'parent_id'. to do this, I will import and use the script I created with a custom function that takes in a dataframe and groups it by a certian column and then applies a specified aggreating function.

In [12]:
# import the custom script
import sample_script as ps

# run it on the data
themes_with_parents_group = ps.custom_agg(themes_with_parents, 'name', 'parent_id')
themes_with_parents_group

,name,count
0,12V,2
1,4 Juniors,1
2,4.5V,2
3,9V,2
4,Advent,1
...,...,...
314,World City,2
315,World Racers,1
316,X-Men,1
317,Xalax,1


lets do a scatter plot of the themes with the most parents.

In [13]:
# first sort the counts and extract the top 20
top_30_parents = themes_with_parents_group.sort_values(by=["count"], ascending=False)[
    :30
]

# now plot it using altair
top_30_parents_plot = (
    alt.Chart(top_30_parents, width=500, height=300)
    .mark_circle()
    .encode(
        x=alt.X("name:O", sort="y", title="Theme name"),
        y=alt.Y("count:Q", title="Frequency of parent"),
    )
    .properties(title="Top 30 most frequent themes with parents")
)
top_30_parents_plot

alt.Chart(...)

It seems that the **supplemental** theme has the most frequent parent theme. This is expected because most themes have a supplemental pack with extra pieces if needed. 

Now it's time to answer the original question. Which theme has the most sets? To do this, I will first create a dataframe that contains both the theme `id` and the `counts` of the occurence of said `id`.

In [14]:
# using the themes table, get a count of the occurences of each of the theme id
theme_counts = sets["theme_id"].value_counts()
theme_counts.head()

501    246
371    240
504    219
220    216
494    192
Name: theme_id, dtype: int64

This is a pandas series. Lets transform it in to a dataframe so it becomes easier to work it.

In [15]:
# transform the frequency counts in to a dataframe
theme_counts_df = pd.DataFrame({"id": theme_counts.index, "count": theme_counts.values})
theme_counts_df.head()

,id,count
0,501,246
1,371,240
2,504,219
3,220,216
4,494,192


Perfect! Now that I have it in a dataframe, I can merge it with the orginal **themes** dataframe which will enable  the computation of the LEGO themes with the most sets.

In [16]:
set_theme_merged = pd.merge(theme_counts_df, themes, on="id")
set_theme_merged

,id,count,name,parent_id
0,501,246,Gear,NaN
1,371,240,Supplemental,365.0
2,504,219,Duplo,NaN
3,220,216,City,217.0
4,494,192,Friends,NaN
...,...,...,...,...
570,45,1,Cargo,38.0
571,612,1,Star Wars Episode 8,158.0
572,508,1,4 Juniors,507.0
573,444,1,Adventurers,443.0


I am not interested in looking at rows with `parent_id` since this provides no information about themes with the most sets. Lets remove them.

In [17]:
# use the .isnull() function to get the rows with null values in 'parent_id'
set_theme_merged_no_parent = set_theme_merged[pd.isnull(set_theme_merged["parent_id"])]
set_theme_merged_no_parent

,id,count,name,parent_id
0,501,246,Gear,NaN
2,504,219,Duplo,NaN
4,494,192,Friends,NaN
5,435,191,Ninjago,NaN
6,443,185,Service Packs,NaN
...,...,...,...,...
545,475,1,Western,NaN
547,258,1,Mindstorms,NaN
559,560,1,Universe,NaN
560,608,1,Disney,NaN


Lets extract the top `20` themes with the most sets.

In [18]:
top_20_themes = set_theme_merged_no_parent.sort_values(by=["count"], ascending=False)[:20]
top_20_themes

,id,count,name,parent_id
0,501,246,Gear,NaN
2,504,219,Duplo,NaN
4,494,192,Friends,NaN
5,435,191,Ninjago,NaN
6,443,185,Service Packs,NaN
7,1,172,Technic,NaN
8,22,148,Creator,NaN
12,254,125,Bulk Bricks,NaN
15,158,105,Star Wars,NaN
23,408,90,LEGO Brand Store,NaN


It seems that in fact it is **Gear** that is the theme with the most sets. This is very different from my initial assumption of it being **Star Wars**. Lets generate a bar plot to visualize this.

In [19]:
# Visualize the top 20 themes with the most sets using a bar plot.
top_20_plot = (
    alt.Chart(top_20_themes, width=500, height=300)
    .mark_bar()
    .encode(
        x=alt.X("name:N", title="LEGO Theme", sort="-y"),
        y=alt.Y("count:Q", title="Count of the number of sets"),
    )
    .properties(title="Number of sets for LEGO theme")
)
top_20_plot

alt.Chart(...)

# Discussions

In this work, I analyzed the LEGO dataset and tried to compute which theme had the most sets. Before answering this question, I did some exploratory data analysis to see how the average number of LEGO parts changed over the year. Generally, there is an increasing trend in the average number of LEGO parts over the years. However, there are a few outliers and pockets where the average number of parts decreased drastically and then increased. I also found out that the supplementary theme was part of the most sets which is not unexpected. 

It is quite surprising that **Gear** is the with the most LEGO sets. My guess would have been **Star Wars**. However, **Star Wars** only has about $110$ sets, which is small relative to how popular it has become. Especially with so many LEGO movie adaptations and video games that have been released over the years, it is surprising that **Star Wars** does not make the top $5$ themes with the most sets.

Another question that could be looked at given this dataset is the change in the distribution of colors over the years. One could look at which color has changed in abundance significantly over the years. This is interesting because LEGO sets come in a variety of different colors. If however, some colors have become more abundant and others less abundant over the years. This change in abundance might imply a change in marketing strategy where the manufacturer made more pieces of a particular color because said color sold more sets than other colors.

# References

Not all the work in this notebook is original. Some parts were borrowed from online resources. I take no credit for parts that are not mine. They were soley used for illustration purposes. Lets give to **Ceasar** what belongs to **Ceasar**.

## Resources used
* [Data Source](https://www.kaggle.com/rtatman/lego-database)
    * This LEGO database used in this work was curated by **Rachel Tatman**.
* [Data Visualization](https://www.kaggle.com/asindico/data-exploration)
    * Inspiration for generating the plotting the average number of parts over the years was taken from **Andrea Sandico**.
* [Question Of Interest](https://www.kaggle.com/nathanlauga/lego-which-are-the-themes-with-the-most-sets)
    * The question of interest was inspired by **Nathan Lauga**.

![](src/Batman.jpg)

Image obtained from [here](https://www.google.com/search?q=im+batman+lego&tbm=isch&ved=2ahUKEwj7ra-zserrAhVSlJ4KHcsfDYcQ2-cCegQIABAA&oq=im+&gs_lcp=CgNpbWcQARgAMgQIIxAnMgQIIxAnMgQIABBDMgQIABBDMgQIABBDMgQIABBDMgQIABBDMgQIABBDMgQIABBDMgQIABBDOgcIABCxAxBDOgUIABCxAzoCCABQqa0BWMWwAWD8uwFoAHAAeACAAUOIAbQBkgEBM5gBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=ZEtgX_vrNNKo-gTLv7S4CA&bih=932&biw=1920#imgrc=141ME2CrunlATM)